# Connect to Microsoft Access Database file
## Requirements 
- In order to access the db x64 systems must have the `Microsoft Access Driver (*.mdb, *.accdb)` driver 
- `pyodbc`
  - After installing pyodbc (e.g. `pip install pyodbc`) run:
   `pyodbc.drivers()` to checkout your system drivers.
- Refactor the `ArchiveIndex_Beta_Install.mdb` file path according to your local setup

In [2]:
import pyodbc
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=Standalone_GCBM\\input_database\\ArchiveIndex_Beta_Install.mdb;')
cursor = conn.cursor()
print('Connected')

Connected


### This script converts the encoding of future outputs in utf-16 for convinience

In [3]:
def decode_sketchy_utf16(raw_bytes):
    s = raw_bytes.decode("utf-16le", "ignore")
    try:
        n = s.index('\u0000')
        s = s[:n]  # respect null terminator
    except ValueError:
        pass
    return s

prev_converter = conn.get_output_converter(pyodbc.SQL_WVARCHAR)
conn.add_output_converter(pyodbc.SQL_WVARCHAR, decode_sketchy_utf16)

# Table Columns & Column Types

In [4]:
tables = []

for table in cursor.tables(tableType='TABLE'):
    tables.append(table.table_name)

for row in tables:
    print(f"Table {row} with columns:")
    for col in cursor.columns(row):
        print(f"{col.column_name} : {col.type_name}")

Table Paste Errors with columns:
SPUID : DOUBLE
B1 : DOUBLE
B2 : DOUBLE
Table tblAdminBoundaryDefault with columns:
AdminBoundaryID : SMALLINT
AdminBoundaryName : VARCHAR
SoftwoodTopProportion : REAL
SoftwoodStumpProportion : REAL
HardwoodTopProportion : REAL
HardwoodStumpProportion : REAL
Table tblAfforestationPreTypeDefault with columns:
PreTypeID : INTEGER
Name : VARCHAR
Description : VARCHAR
Table tblBiomassComponent with columns:
BiomassID : SMALLINT
BiomassComponentName : VARCHAR
Table tblBiomassToCarbonDefault with columns:
BiomassID : SMALLINT
Softwood : BIT
Multiplier : REAL
Table tblBiomassToHeightParameterDefault with columns:
SPUID : INTEGER
B1 : REAL
B2 : REAL
Table tblBioTotalStemwoodForestTypeDefault with columns:
DefaultSPUID : SMALLINT
DefaultForestTypeID : SMALLINT
A : DOUBLE
B : DOUBLE
a_nonmerch : DOUBLE
b_nonmerch : DOUBLE
k_nonmerch : DOUBLE
cap_nonmerch : DOUBLE
a_sap : DOUBLE
b_sap : DOUBLE
k_sap : DOUBLE
cap_sap : DOUBLE
a1 : DOUBLE
a2 : DOUBLE
a3 : DOUBLE
b1 :

# View Columns & Column Types

In [5]:
views = []

for view in cursor.tables(tableType='VIEW'):
    views.append(view.table_name)

for view in views:
    print(f"View {view} with columns:")
    for col in cursor.columns(view):
        print(f"{col.column_name} : {col.type_name}")

View qryCBMRunJoin with columns:
CBMRunID : INTEGER
Name : VARCHAR
Description : LONGCHAR
Author : VARCHAR
Status : INTEGER
StartedAt : DATETIME
CompletedAt : DATETIME
ClientID : INTEGER
ClientName : VARCHAR
InputDBID : INTEGER
InputDB : VARCHAR
CBMVersionID : INTEGER
CBMExecutable : VARCHAR
InputCBMRunID : INTEGER
View qryCombustedProportion with columns:
NIR_version : DOUBLE
DefaultEco : DOUBLE
DMID : DOUBLE
DMRow : DOUBLE
SumOfProportion : DOUBLE
View qrySimulationJoin with columns:
SimulationID : INTEGER
Name : VARCHAR
Description : LONGCHAR
Author : VARCHAR
Status : INTEGER
StartedAt : DATETIME
CompletedAt : DATETIME
CBMRunID : INTEGER
CBMRunName : VARCHAR
StandInitializationID : INTEGER
StandInitializationName : VARCHAR
InputSimulationID : INTEGER
InputDB : VARCHAR
ResultsDB : VARCHAR
CBMInputFilesPath : VARCHAR
CBMOutputFilesPath : VARCHAR
ResultsPermArch : VARCHAR
RulesVersionName : VARCHAR
AllSPUsProcessed : BIT
IsArchived : BIT
IsInCombinedArchive : BIT
View qryStandInitializ

# Sample Query
- It uses pandas to store the sql output in a `pandas.Dataframe`

In [6]:
import pandas as pd
query = "SELECT * FROM qryCombustedProportion"
res = pd.read_sql(query,conn)
print(res)

# Export to csv
res.to_csv('results.csv',index=False)

     NIR_version  DefaultEco   DMID  DMRow  SumOfProportion
0         2009.0         4.0  281.0    2.0         1.000000
1         2009.0         4.0  281.0    3.0         0.250000
2         2009.0         4.0  281.0    4.0         0.250000
3         2009.0         4.0  281.0    6.0         0.191377
4         2009.0         4.0  281.0    8.0         0.996737
..           ...         ...    ...    ...              ...
635       2010.0        18.0  344.0   12.0         0.138842
636       2010.0        18.0  344.0   13.0         1.000000
637       2010.0        18.0  344.0   15.0         0.614602
638       2010.0        18.0  344.0   17.0         0.163778
639       2010.0        18.0  344.0   18.0         0.284664

[640 rows x 5 columns]
